##  SETUP VIRTUAL MACHINES FOR MULTIPLE USERS
Use Azure Virtual Machine Scale Set (VMSS) and Azure Command Line Interface (Az CLI) to create N virtual machines. After setting up we invoke a setup script on each VM to setup the environment. 



## Azure Resource Setup

In [1]:
AZURE_SUBSCRIPTION_ID = "your-subscription-id"
LOCATION = "your-resource-location"  # e.g. australiaeast
VM_SIZE = "your-vm-size"  # e.g. Standard_D8_v3
RG_NAME = "your-resource-group-name"
VMSS_NAME = "{}-vmss".format(RG_NAME)

# Number of VM instances in a scale set 
NUM_VM = 3

In [8]:
AZURE_SUBSCRIPTION_ID = "9086b59a-02d7-4687-b3fd-e39fa5e0fd9b"
LOCATION = "westus"  # e.g. australiaeast
VM_SIZE = "Standard_D8_v3"  # e.g. Standard_D8_v3
RG_NAME = "emawamlads"
VMSS_NAME = "{}-vmss".format(RG_NAME)

# Number of VM instances in a scale set 
NUM_VM = 3

In [4]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DBACTW85V to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "id": "bc4170f0-cc6e-49d2-ba65-bc00a7a4df6b",
    "isDefault": false,
    "name": "AG-AzureCAT-AIAbhinav-Test-COGSNonProd-IO1685734",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "emawa@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "id": "7bdec0cd-b80c-4715-8798-c694ca75dd05",
    "isDefault": false,
    "name": "Visual Studio Enterprise",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "emawa@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "id": "a18897a6-7e44-457d-9260-f2854c0aca42",
    "isDefault": false,
    "name": "Azure SDK Engineering System",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd

In [5]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

In [6]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "9086b59a-02d7-4687-b3fd-e39fa5e0fd9b",
  "isDefault": true,
  "name": "AICAT-VB-NLP",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "emawa@microsoft.com",
    "type": "user"
  }
}


### Create VMSS

In [9]:
!az group create --name {RG_NAME} --location {LOCATION}

{
  "id": "/subscriptions/9086b59a-02d7-4687-b3fd-e39fa5e0fd9b/resourceGroups/emawamlads",
  "location": "westus",
  "managedBy": null,
  "name": "emawamlads",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [18]:
# Setup public-ip for each vm by using `--public-ip-per-vm` parameter
!az vmss create -g {RG_NAME} -n {VMSS_NAME} --instance-count {NUM_VM} --image microsoft-dsvm:linux-data-science-vm-ubuntu:linuxdsvmubuntu:latest --vm-sku {VM_SIZE} --public-ip-per-vm --admin-username vmadmin --admin-password nlpbpMlads2019

{- Finished ..
  "vmss": {
    "doNotRunExtensionsOnOverprovisionedVMs": false,
    "overprovision": true,
    "provisioningState": "Succeeded",
    "singlePlacementGroup": true,
    "uniqueId": "7b8ed921-9dfa-43ab-99a6-59024f6737d4",
    "upgradePolicy": {
      "mode": "Manual"
    },
    "virtualMachineProfile": {
      "networkProfile": {
        "networkInterfaceConfigurations": [
          {
            "name": "emawa36caNic",
            "properties": {
              "dnsSettings": {
                "dnsServers": []
              },
              "enableAcceleratedNetworking": false,
              "enableIPForwarding": false,
              "ipConfigurations": [
                {
                  "name": "emawa36caIPConfig",
                  "properties": {
                    "loadBalancerBackendAddressPools": [
                      {
                        "id": "/subscriptions/9086b59a-02d7-4687-b3fd-e39fa5e0fd9b/resourceGroups/emawamlads/providers/Microsoft.Network/loadBa

In [19]:
# Check public ip addresses of the instances
!az vmss list-instance-public-ips --resource-group {RG_NAME} --name {VMSS_NAME} | grep 'ipAddress'

    "ipAddress": "40.78.101.125",
    "ipAddress": "40.78.101.167",
    "ipAddress": "40.78.101.176",


## Create User Accounts and Environment Setup

In [13]:
SCRIPT = "setup_vm.sh"

In [15]:
%%writefile setup_vm.sh
cd ~  

# clone repo and install the conda env 
git clone https://github.com/microsoft/nlp-recipes.git

# change permission as we copy this into each user's folder
chmod -R ugo+rwx /root/nlp-recipes

source /data/anaconda/etc/profile.d/conda.sh

cd /root/nlp-recipes
python tools/generate_conda_file.py
conda env create -n nlp_cpu -f nlp_cpu.yaml

conda activate my_env_name
python -m ipykernel install --user --name nlp_cpu --display-name "Python (nlp_cpu)"

# Pip install the utils_nlp package in the conda environment
pip install -e . 


# add users to jupyterhub
echo 'c.Authenticator.whitelist = {"mlads1", "mlads2", "mlads3", "mlads4", "mlads5", "mlads6", "mlads7", "mlads8", "mlads9", "mlads10", "mlads11", "mlads12", "mlads13", "mlads14", "mlads15", "mlads16", "mlads17", "mlads18", "mlads19", "mlads20", "mlads21", "mlads22", "mlads23", "mlads24", "mlads25", "mlads26", "mlads27", "mlads28", "mlads29", "mlads30", "mlads31", "mlads32"}' | sudo tee -a /etc/jupyterhub/jupyterhub_config.py

# create the users on the vm 
for i in {1..32}
do
    USERNAME=mlads$i
    PASSWORD=nlpmlads$i
    sudo adduser --quiet --disabled-password --gecos "" $USERNAME
    echo "$USERNAME:$PASSWORD" | sudo chpasswd
    rm -rf /data/home/$USERNAME/notebooks/*
    
    # copy repo
    cp -ar /root/nlp-recipes /data/home/$USERNAME/notebooks
done

# restart jupyterhub 
sudo systemctl stop jupyterhub 
sudo systemctl start jupyterhub 

exit

Overwriting setup_vm.sh


In [20]:
!az vmss list-instances -n {VMSS_NAME} -g {RG_NAME} --query "[].id" --output tsv | az vmss run-command invoke --command-id RunShellScript --scripts @{SCRIPT} --ids @-

[- Finished ..
  {
    "value": [
      {
        "code": "ProvisioningState/succeeded",
        "displayStatus": "Provisioning succeeded",
        "level": "Info",
        "message": "Enable succeeded: \n[stdout]\nGenerated conda file: nlp_cpu.yaml\n\nTo create the conda environment:\n$ conda env create -f nlp_cpu.yaml\n\nTo update the conda environment:\n$ conda env update -f nlp_cpu.yaml\n\nTo register the conda environment in Jupyter:\n$ conda activate nlp_cpu\n$ python -m ipykernel install --user --name nlp_cpu --display-name \"Python (nlp_cpu)\"\n\nInstalled kernelspec nlp_cpu in /root/.local/share/jupyter/kernels/nlp_cpu\nObtaining file:///root/nlp-recipes\n  Installing build dependencies: started\n  Installing build dependencies: finished with status 'done'\n  Getting requirements to build wheel: started\n  Getting requirements to build wheel: finished with status 'done'\n    Preparing wheel metadata: started\n    Preparing wheel metadata: finished with status 'done'\nc.Authent